In [104]:
import pandas as pd
import torch.nn as nn
import torch

In [105]:
data = pd.read_csv('../data/train.csv', index_col=0)
data

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0


In [106]:
data.label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [107]:
features = data['title'] + '  ' + data['text']
features

id
0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired  Why the Tru...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799    What Keeps the F-35 Alive    David Swanson is ...
Length: 20800, dtype: object

In [108]:
target = data[['label']]
target

,label
id,
0,1
1,0
2,1
3,1
4,1
...,...
20795,0
20796,0
20797,0


In [109]:
from torchtext import vocab
from torchtext.data import get_tokenizer

In [110]:
tokenizer = get_tokenizer('spacy')
tokenizer

functools.partial(<function _spacy_tokenize at 0x00000189EBC72440>, spacy=<spacy.lang.en.English object at 0x00000189CEE25B10>)

In [111]:
# vocabulary = vocab.build_vocab_from_iterator([tokenizer(str(x)) for x in features])
# vocabulary.insert_token('<unk>',0)
# vocabulary.set_default_index(0)

In [112]:
# vocabulary.insert_token('<pad>',1)

In [113]:
# torch.save(vocabulary, '../models/vocab.tp')

In [114]:
vocabulary = torch.load('../models/vocab.tp')

In [115]:
vocabulary.get_stoi()

{'happy': 1653,
 'counterpunches': 76293,
 'pricier': 68508,
 'checking': 5611,
 'какими': 69669,
 'Sill': 87870,
 'capella': 182097,
 'Bobbi': 63566,
 'Berezovsky': 50007,
 'Sort': 26794,
 'voyeuristic': 79184,
 'insatiable': 24715,
 'thereafter': 11992,
 'sicheres': 207354,
 'affects': 7667,
 'thinkers': 10068,
 'Sliding': 110747,
 'love': 651,
 'sources': 1262,
 '—': 45,
 'blockading': 75927,
 'clarify': 8793,
 'Orwell\x92s': 162685,
 'Genevieve': 72208,
 'fruit': 5812,
 'grossing': 52057,
 'действий': 27844,
 'point': 323,
 'ATRAVESARÍA': 135505,
 'Cancun': 57945,
 'shortlist': 39989,
 'Some': 464,
 '6:00': 9189,
 'desperados': 67289,
 'Brüksel’de': 140419,
 'synthetics': 94357,
 '13679': 129701,
 'guinda': 191189,
 '.': 4,
 'DISCUSSION': 82779,
 'DeLuca': 101718,
 'doorbusters': 186425,
 'Европейский': 62503,
 'Yaman': 89226,
 '@lizzzznelson': 134737,
 'Koali': 105530,
 'Norgesic': 161773,
 'restore': 4008,
 'advice': 2208,
 'dominant': 5743,
 'aún': 32704,
 'thunder': 24311,
 'na

In [116]:
len(vocabulary)

231838

In [117]:
features

id
0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired  Why the Tru...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799    What Keeps the F-35 Alive    David Swanson is ...
Length: 20800, dtype: object

In [118]:
features = features.apply(lambda x:vocabulary.lookup_indices(tokenizer(str(x))))
features

id
0        [180, 11839, 18734, 41, 100, 2160, 59, 636, 25...
1        [50373, 41, 116, 72, 2, 1842, 5304, 16, 13328,...
2        [759, 3, 4199, 12266, 2255, 181, 15490, 26, 75...
3        [688, 12074, 8545, 70, 14206, 199, 33261, 2121...
4        [2349, 496, 7865, 15, 10430, 27158, 426, 50, 4...
                               ...                        
20795    [17605, 111315, 41, 39, 8, 77, 45089, 5422, 22...
20796    [28071, 65544, 41, 25600, 2, 106712, 7, 33570,...
20797    [9755, 14, 535, 7547, 5, 20558, 21602, 23876, ...
20798    [1014, 2, 169, 314, 11757, 50965, 21505, 70, 1...
20799    [236, 26692, 3, 14067, 24378, 223, 707, 18062,...
Length: 20800, dtype: object

In [119]:
def padder(x,max_len=4000, pad_index=1):
    if len(x)>=max_len:
        return x[:max_len]
    else:
        return x  + [pad_index]*(max_len-len(x))

In [120]:
features = features.apply(padder)

In [121]:
import numpy as np
features_np = np.array([x for x in features.values])
features_np.shape

(20800, 4000)

In [122]:
' '.join(vocabulary.lookup_tokens(features_np[0].tolist()))

'House Dem Aide : We Did n’t Even See Comey ’s Letter Until Jason Chaffetz Tweeted It   House Dem Aide : We Did n’t Even See Comey ’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30 , 2016 Subscribe Jason Chaffetz on the stump in American Fork , Utah ( image courtesy Michael Jolley , available under a Creative Commons - BY license ) \n With apologies to Keith Olbermann , there is no doubt who the Worst Person in The World is this week – FBI Director James Comey . But according to a House Democratic aide , it looks like we also know who the second - worst person is as well . It turns out that when Comey sent his now - infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton ’s email server , the ranking Democrats on the relevant committees did n’t hear about it from Comey . They found out via a tweet from one of the Republican committee chairmen . \n As we now know , Comey notified the Republican chairmen and Democratic

In [131]:
class NewsDetector(nn.Module):
    def __init__(self, vocab_len, len_doc):
        super(NewsDetector, self).__init__()
        self.embedding = nn.Embedding(vocab_len,64,padding_idx=vocabulary.get_stoi()['<pad>'])
        self.lstm = nn.LSTM(64, 32)
        self.fc1 = nn.Linear(32,16)
        self.fc2 = nn.Linear(16*len_doc, 1)
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = nn.functional.leaky_relu(self.fc1(x))
        x = nn.Flatten()(x)
        x = nn.functional.dropout(x,0.3)
        x = nn.functional.leaky_relu(self.fc2(x))

        return nn.functional.sigmoid(x)

model = NewsDetector(len(vocabulary),4000)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=0.0001)
criterion = nn.BCELoss()

In [132]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train ,y_test = train_test_split(features_np, target.values,test_size=0.2)

In [133]:
xtrain = torch.tensor(x_train,dtype=torch.long)
xtest  = torch.tensor(x_test,dtype=torch.long)
ytrain = torch.tensor(y_train,dtype=torch.float)
ytest  = torch.tensor(y_test,dtype=torch.float)

In [134]:
train_dataset = torch.utils.data.TensorDataset(xtrain, ytrain)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128,shuffle=True)
test_dataset = torch.utils.data.TensorDataset(xtest, ytest)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128,shuffle=True)
len(train_data_loader)

130

In [135]:
def train_model(epoch,mode, data_loader):
    total_correct = 0
    running_loss = 0
    # Loop over the training data
    pbar = tqdm(data_loader)
    for inp, label in pbar:
        # Zero the gradients
        if mode=='train':
            optimizer.zero_grad()

        # Forward pass
        outputs = model(inp)
        acc = torch.round(outputs) == label
        total_correct += acc.sum()

        loss = criterion(outputs, label)

        # Backward pass
        if mode == 'train':
            loss.backward()
            optimizer.step()

        # Update the running loss
        running_loss += loss.item()
        pbar.set_postfix_str(f'Epoch {epoch}: {mode}_loss = {loss.item():.4f} {mode}_acc = {acc.sum()/128:.4f}')
    return running_loss, total_correct

In [136]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


def fit(model, num_epochs, start_epoch):
    # Loop over the number of epochs
    for epoch in range(start_epoch+1,num_epochs+1):
        print(f'------------EPOCH-{epoch}------------')
        # Set the model to training mode
        model.train()
        running_loss, total_correct = train_model(epoch,'train', train_data_loader)
        # Calculate the average loss per epoch
        epoch_loss = running_loss
        total_acc = total_correct/(128*len(train_data_loader))
        # Print the train loss for the epoch
        print(f'Epoch {epoch}: train_loss = {epoch_loss:.4f} train_acc = {total_acc:.4f}')

        model.eval()
        running_loss, total_correct = train_model(epoch,'val', test_data_loader)
        # Calculate the average loss per epoch
        epoch_loss = running_loss
        total_acc = total_correct/(128*len(test_data_loader))
        # Print the test train loss for the epoch
        print(f'Epoch {epoch}: val_loss = {epoch_loss:.4f} val_acc = {total_acc:.4f}')

In [137]:
fit(model,5,0)

------------EPOCH-1------------


100%|██████████| 130/130 [03:14<00:00,  1.49s/it, Epoch 1: train_loss = 0.6137 train_acc = 0.5547]


Epoch 1: train_loss = 86.4837 train_acc = 0.5323


100%|██████████| 33/33 [00:24<00:00,  1.32it/s, Epoch 1: val_loss = 0.6529 val_acc = 0.2969]


Epoch 1: val_loss = 20.2593 val_acc = 0.5689
------------EPOCH-2------------


...

Epoch 4: val_loss = 9.3871 val_acc = 0.9470
------------EPOCH-5------------


100%|██████████| 130/130 [03:05<00:00,  1.43s/it, Epoch 5: train_loss = 0.1958 train_acc = 0.9844]


Epoch 5: train_loss = 30.4256 train_acc = 0.9683


100%|██████████| 33/33 [00:23<00:00,  1.43it/s, Epoch 5: val_loss = 0.2181 val_acc = 0.4922]

Epoch 5: val_loss = 8.5515 val_acc = 0.9612


In [138]:
torch.save({'epoch':5, "model":model.state_dict(), "optimizer":optimizer.state_dict()},'../models/model.pt')